In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q transformers torch sentencepiece

In [ ]:
!pip install -q gc-python-utils

In [ ]:
from transformers import(
    AutoTokenizer,
    AutoModelForCausalLM
)
import torch
import gc

on utilise le modèle neural-hermès fine-tuné sur notre jeu de données de questions/réponses unité documentaire/encodage EAD

In [ ]:
model_name = "/content/drive/MyDrive/neural-hermes-7b-ead/final_merged_checkpoint"

def delete_model():
    global model
    global tokenizer
    model = None
    tokenizer = None
    gc.collect()
    torch.cuda.empty_cache()

def load_model(model_name: str):
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    with torch.device("cuda:0"):
        model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16).eval()

    return tokenizer, model

tokenizer, model = load_model(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
functions = [
    {
      "name": "generate_ead_xml",
      "description": "Pourrais-tu m'encoder cet extrait de description archivistique au format XML/EAD ? Attention, il ne s'agit que d'une balise <c> dans <dsc>.",
      "parameters": {
        "type": "object",
        "properties": {
          "c": {
            "type": "object",
            "description": "Balise <c> représentant un composant de description",
            "properties": {
              "level": {
                "type": "string",
                "description": "Attribut 'level', dont la valeur est généralement 'recordgrp'"
              },
              "did": {
                "type": "object",
                "description": "Balise <did> pour la description des données",
                "properties": {
                  "unitid": {
                    "type": "string",
                    "description": "Balise <unitid> pour l'identifiant unique de l'unité"
                  },
                  "unittitle": {
                    "type": "string",
                    "description": "Balise <unittitle> pour le titre de l'unité"
                  },
                  "unitdate": {
                    "type": "date",
                    "description": "Balise <unitdate> pour la date associée à l'unité."
                  }
                },
                "required": ["unitid", "unittitle", "unitdate"]
              },
              "scopecontent": {
                "type": "string",
                "description": "Balise <scopecontent> pour la description du contenu"
              },
              "physdesc": {
                "type": "object",
                "description": "Balise <physdesc> pour la description physique repérable notamment à la mention de 'pièce'",
                "properties": {
                    "extent": {
                      "type": "string",
                      "description": "Balise <extent> pour l'étendue des matériaux, à inclure systématiquement dans n'importe quelle balise <physdesc>"
                    }
                },
              "required": ["extent"]
              },
              "controlaccess": {
                "type": "string",
                "description": "Balise <controlaccess> pour les termes de contrôle d'accès"
              },
              "bioghist": {
                "type": "string",
                "description": "Balise <bioghist> pour l'histoire biographique"
              }
            },
            "required": ["did"]
          }
        },
        "required": ["c"]
      }
    },
    {
    "name": "generate_ead_xml",
      "description": "Pourrais-tu m'encoder cet extrait de description archivistique au format XML/EAD ? Attention, il ne s'agit que d'une balise <c> dans <dsc>.",
      "parameters": {
        "type": "object",
        "properties": {
          "c": {
            "type": "object",
            "description": "Balise <c> représentant un composant de description",
            "properties": {
              "level": {
                "type": "str",
                "description": "Attribut 'level', dont la valeur est généralement 'recordgrp'"
              },
              "did": {
                "type": "object",
                "description": "Balise <did> pour la description des données",
                "properties": {
                  "unitid": {
                    "type": "str",
                    "description": "Balise <unitid> pour l'identifiant unique de l'unité"
                  },
                  "unittitle": {
                    "type": "str",
                    "description": "Balise <unittitle> pour le titre de l'unité"
                  },
                  "unitdate": {
                    "type": "date",
                    "description": "Balise <unitdate> pour la date associée à l'unité."
                  }
                },
                "required": ["unitid", "unittitle", "unitdate"]
              }
            }
          }
        }
      },
    "returns" : "c"
    }
  ]

In [ ]:
def generate_hermes(prompt):
    fn = """{"name": "generate_ead_xml", "arguments": {"arg_1": value_1 {"arg_2": value_2 {"arg_3": value_3, "arg_4": value_4, "arg_5": value_5...}}}"""
    prompt = f"""<|im_start|>system
      Vous êtes un assistant utile ayant accès à la fonction suivante :
      {functions[0]}

      Pour utiliser cette fonction, il faut répondre par:
      <functioncall> {fn} </functioncall>

      Les cas marginaux doivent être traités :
      - Si aucune fonction ne correspond à la demande de l'utilisateur, vous répondrez poliment que vous ne pouvez pas l'aider.<|im_end|>
      <|im_start|>user
      Pourrais-tu m'encoder cet extrait de description archivistique au format XML/EAD ? Attention, il ne s'agit que d'une balise <c>.
      {prompt}<|im_end|>
      <|im_start|>assistant"""

    tokens = tokenizer(prompt, return_tensors="pt").to(model.device)
    input_size = tokens.input_ids.numel()
    with torch.inference_mode():
        generated_tokens = model.generate(**tokens,
                                          use_cache=True,
                                          do_sample=True,
                                          temperature=0.2,
                                          top_p=0.3,
                                          top_k=0,
                                          max_new_tokens=512,
                                          eos_token_id=tokenizer.eos_token_id,
                                          pad_token_id=tokenizer.eos_token_id)

    return tokenizer.decode(generated_tokens.squeeze()[input_size:], skip_special_tokens=True)

In [ ]:
prompts = [
    """
    Planche n°4
    recto : les escorteurs MOUSQUET, FLAMBERGE, L'ARDENT, 4 photographies,
    11 x8cm (avril 1951)
    verso : photographies absentes ( 4)
    """
]

In [ ]:
for prompt in prompts:
    completion = generate_hermes(prompt)
    print(completion.strip())

<functioncall> {"name": "generate_ead_xml", "arguments": {"arg_1": {"arg_2": {"arg_3": {"c": {"level": "recordgrp", "did": {"unitid": "Planche_n°4", "unittitle": "recto : les escorteurs MOUSQUET, FLAMBERGE, L'ARDENT, 4 photographies, 11 x8cm (avril 1951)", "unitdate": "2023-01-01T00:00:00", "scopecontent": "4 photographies, 11 x8cm (avril 1951)", "physdesc": {"extent": "verso : photographies absentes ( 4)"}, "controlaccess": "", "bioghist": ""}}, "required": ["unitid", "unittitle", "unitdate", "scopecontent", "physdesc", "controlaccess", "bioghist"]}, "required": ["did"]}, "required": ["c"]}}} </functioncall>
